In [1]:
import pandas as pd
import numpy as np
import datetime

#%matplotlib inline

# Cargo CSV

In [2]:
test_final = pd.read_csv('../data/test_final_100k.csv')
test_final = test_final.drop('id',axis=1)

In [3]:
postulantes_genero_y_edad = pd.read_csv('../data/datos_navent_fiuba/fiuba_2_postulantes_genero_y_edad.csv')
postulantes_genero_y_edad = postulantes_genero_y_edad.append(pd.read_csv('../data/fiuba_hasta_15_abril/fiuba_2_postulantes_genero_y_edad.csv'))
postulantes_genero_y_edad = postulantes_genero_y_edad.append(pd.read_csv('../data/fiuba_desde_15_abril/fiuba_2_postulantes_genero_y_edad.csv'))
postulantes_genero_y_edad = postulantes_genero_y_edad.reset_index().drop('index',axis=1)

In [4]:
vistas = pd.read_csv('../data/datos_navent_fiuba/fiuba_3_vistas.csv')
vistas = vistas.append(pd.read_csv('../data/fiuba_hasta_15_abril/fiuba_3_vistas.csv'))
vistas = vistas.append(pd.read_csv('../data/fiuba_desde_15_abril/fiuba_3_vistas.csv'))
vistas = vistas.reset_index().drop('index',axis=1)

In [5]:
postulaciones = pd.read_csv('../data/datos_navent_fiuba/fiuba_4_postulaciones.csv')
postulaciones = postulaciones.append(pd.read_csv('../data/fiuba_hasta_15_abril/fiuba_4_postulaciones.csv'))
# no tiene postulaciones -> pd.read_csv('data/fiuba_desde_15_abril/')
postulaciones = postulaciones.reset_index().drop('index',axis=1)

In [6]:
avisos_detalle = pd.read_csv('../data/datos_navent_fiuba/fiuba_6_avisos_detalle.csv')
avisos_detalle = avisos_detalle.append(pd.read_csv('../data/fiuba_hasta_15_abril/fiuba_6_avisos_detalle.csv'))
avisos_detalle = avisos_detalle.append(pd.read_csv('../data/fiuba_desde_15_abril/fiuba_6_avisos_detalle.csv'))
avisos_detalle = avisos_detalle.append(pd.read_csv('../data/fiuba_6_avisos_detalle_missing_nivel_laboral.csv'))
avisos_detalle = avisos_detalle.reset_index().drop('index',axis=1)

In [7]:
avisos_detalle = avisos_detalle.drop(['ciudad','mapacalle','nombre_zona','descripcion','titulo','tipo_de_trabajo','nivel_laboral','denominacion_empresa','idpais'],axis=1)

In [8]:
avisos_detalle = avisos_detalle.drop_duplicates(subset=['idaviso'],keep='last')
avisos_detalle = avisos_detalle.reset_index().drop('index',axis=1)

# Preproceso datos

### Limpio el DF *postulantes_genero_y_edad*

Luego veremos que hacer con los ids sin fecha de nacimiento, por el momento los dejamos.

In [9]:
postulantes_genero_y_edad = postulantes_genero_y_edad.drop_duplicates()

In [10]:
postulantes_genero_y_edad[postulantes_genero_y_edad['idpostulante'] == '0zP6MJa']

,idpostulante,fechanacimiento,sexo
168364,0zP6MJa,1985-08-16,FEM
525452,0zP6MJa,1989-08-16,FEM


In [11]:
postulantes_genero_y_edad[postulantes_genero_y_edad['idpostulante'] == '0zPvY49']

,idpostulante,fechanacimiento,sexo
485636,0zPvY49,1991-03-26,MASC
678829,0zPvY49,1991-03-26,FEM


In [12]:
#Dejo la ultima aparicion del duplicado, supongo que es el update más reciente
postulantes_genero_y_edad = postulantes_genero_y_edad.drop_duplicates(subset=['idpostulante'],keep='last')

In [13]:
postulantes_genero_y_edad[postulantes_genero_y_edad['idpostulante'] == '0zP6MJa']

,idpostulante,fechanacimiento,sexo
525452,0zP6MJa,1989-08-16,FEM


In [14]:
postulantes_genero_y_edad[postulantes_genero_y_edad['idpostulante'] == '0zPvY49']

,idpostulante,fechanacimiento,sexo
678829,0zPvY49,1991-03-26,FEM


In [15]:
postulantes_genero_y_edad['sexo'].value_counts()

FEM           251205
MASC          227853
NO_DECLARA     25342
0.0                7
Name: sexo, dtype: int64

Al igual que con la fecha de nacimiento, por el momento, dejamos los ids con sexo 0.0

In [16]:
postulantes_genero_y_edad = postulantes_genero_y_edad.reset_index().drop('index',axis=1)

### Limpio el DF *vistas*

In [26]:
vistas.head()

,idAviso,timestamp,idpostulante
0,1111780242,2018-02-23T13:38:13.187-0500,YjVJQ6Z
1,1112263876,2018-02-23T13:38:14.296-0500,BmVpYoR
2,1112327963,2018-02-23T13:38:14.329-0500,wVkBzZd
3,1112318643,2018-02-23T13:38:17.921-0500,OqmP9pv
4,1111903673,2018-02-23T13:38:18.973-0500,DrpbXDP


In [17]:
vistas.isnull().sum()

idAviso         0
timestamp       0
idpostulante    0
dtype: int64

In [18]:
vistas.shape

(18368813, 3)

In [19]:
vistas = vistas.drop_duplicates(keep='last')

In [20]:
vistas = vistas.reset_index().drop('index',axis=1)

In [21]:
vistas.shape

(16754298, 3)

In [22]:
#Se me ocurrió que un postulante pudo ver un aviso mas de una vez, por lo que quizas no se eliminó con
#el anterior drop_duplicates. Creo  que con lo de abajo si los limpio pero no estoy seguro.
vistas = vistas.drop_duplicates(subset=['idAviso','idpostulante'],keep='last')
vistas = vistas.reset_index().drop('index',axis=1)

In [23]:
vistas.shape

(8597915, 3)

### Limpio el DF *postulaciones*

In [24]:
postulaciones = postulaciones.drop_duplicates(keep='last')

In [25]:
postulaciones = postulaciones.reset_index().drop('index',axis=1)

# Preprocesamiento

In [26]:
# Agrego la edad de los postulantes
postulantes_genero_y_edad['fechanacimiento'] = pd.to_datetime(postulantes_genero_y_edad['fechanacimiento'],errors='coerce')
postulantes_genero_y_edad['edad'] = datetime.datetime.now().year - postulantes_genero_y_edad['fechanacimiento'].dt.year

In [27]:
# Despues veo que hago con los que no tienen edad
postulantes_genero_y_edad.isnull().sum()

idpostulante           0
fechanacimiento    26096
sexo                   0
edad               26096
dtype: int64

In [28]:
# Separa la gente postulada, luego los busco en el DF que junta todos los postulantes
# y me quedo con los opuestos, los no postulados.
# esto me va a servir para samplear
# GENTE NO POSTULADA
gente_no_postulada = postulantes_genero_y_edad[-postulantes_genero_y_edad['idpostulante'].\
                                               isin(postulaciones['idpostulante'].value_counts().index)]
gente_no_postulada = gente_no_postulada.reset_index(drop=True)

In [29]:
gente_no_postulada['sexo'].value_counts()

FEM           71992
MASC          67819
NO_DECLARA    15707
0.0               6
Name: sexo, dtype: int64

In [30]:
# Pocos datos con sexo invalido, los desecho ya que son los no postulados
gente_no_postulada = gente_no_postulada.drop(gente_no_postulada[gente_no_postulada['sexo'] == '0.0'].index)

In [31]:
# GENTE POSTULADA
gente_postulada = postulantes_genero_y_edad[postulantes_genero_y_edad['idpostulante'].\
                                               isin(postulaciones['idpostulante'].value_counts().index)]
gente_postulada = gente_postulada.reset_index(drop=True)

In [32]:
gente_postulada['sexo'].value_counts()

FEM           179213
MASC          160034
NO_DECLARA      9635
0.0                1
Name: sexo, dtype: int64

In [33]:
# Reemplazo el sexo invalido por NO_DECLARA
gente_postulada.loc[gente_postulada['sexo'] == '0.0','sexo'] = 'NO_DECLARA'

In [34]:
# Dropeo los datos invalidos de los no postulados (se podria buscar alguna forma de salvarlos)
gente_no_postulada = gente_no_postulada.dropna()
gente_no_postulada = gente_no_postulada.reset_index(drop=True)
gente_no_postulada['edad'] = gente_no_postulada['edad'].astype('int32')

In [35]:
# En un principio los relleno con la media de los postulantes
gente_postulada.loc[gente_postulada['edad'].isnull(),'edad'] = int(postulantes_genero_y_edad['edad'].mean())
gente_postulada['edad'] = gente_postulada['edad'].astype('int32')

In [36]:
gente_postulada.isnull().sum()

idpostulante           0
fechanacimiento    10074
sexo                   0
edad                   0
dtype: int64

# Funciones de utilidad

In [37]:
# fijarse de balancear el sample con 50% de postulados y no postulados

# Generador de sample
# el id tiene que estar en la columna 0
# Recibe los DFs de idpostulante e idaviso
def generador_de_sample_df(idaviso, idpostulante, n_sample, ciclos):
    idpostulante = idpostulante.drop(columns=idpostulante.columns[1:])
    idaviso = idaviso.drop(columns=idaviso.columns[1:])
    idpostulante['tem'] = 1
    idaviso['tem'] = 1
    sample_append = pd.DataFrame(columns=[idpostulante.columns[0],'tem',idaviso.columns[0]])
    for i in range(0,ciclos):
        sample_idpostulante = idpostulante.sample(n=n_sample,random_state=i)
        sample_idaviso = idaviso.sample(n=n_sample,random_state=i)
        sample_merge = sample_idaviso.merge(sample_idpostulante)
        sample_append = sample_append.append(sample_merge)
    sample_append = sample_append.drop('tem',axis=1)
    sample_append = sample_append.reset_index(drop=True)
    return sample_append

In [38]:
# Recibe las series de idpostulante e idaviso
def generador_de_sample_serie(idaviso, idpostulante, n_sample, ciclos):
    idpostulante = pd.DataFrame(idpostulante)
    idaviso = pd.DataFrame(idaviso)
    idpostulante['tem'] = 1
    idaviso['tem'] = 1
    sample_append = pd.DataFrame(columns=[idpostulante.columns[0],'tem',idaviso.columns[0]])
    for i in range(0,ciclos):
        sample_idpostulante = idpostulante.sample(n=n_sample,random_state=i)
        sample_idaviso = idaviso.sample(n=n_sample,random_state=i)
        sample_merge = sample_idaviso.merge(sample_idpostulante)
        sample_append = sample_append.append(sample_merge)
    sample_append = sample_append.drop('tem',axis=1)
    sample_append = sample_append.reset_index(drop=True)
    return sample_append

In [39]:
def borrar_tuplas_test_del_sample(sample,test_final):
    df_return = sample.merge(test_final,how='left',on=['idaviso','idpostulante'],indicator=True)
    df_return = df_return[df_return['_merge'] == 'left_only']
    df_return = df_return.drop('_merge',axis=1)
    return df_return

In [40]:
def oneHotEncoding(df, columna):
    one_hot = pd.get_dummies(df[columna])
    df.drop(columna, axis=1, inplace=True)
    df = pd.concat([df, one_hot], axis=1)
    return df

# Funciones feature

In [41]:
def feature_postulante_se_postulo_al_aviso(tupla_idaviso_idpostulante,postulaciones):
    df_return = tupla_idaviso_idpostulante.merge(postulaciones,on=['idaviso','idpostulante'],how='left')
    return df_return

In [42]:
def feature_genero_edad(tupla_idaviso_idpostulante,postulantes_genero_y_edad):
    df_return = tupla_idaviso_idpostulante.merge(postulantes_genero_y_edad,on='idpostulante',how='left')
    return df_return

# Gente postulada

In [43]:
tupla_postulada = postulaciones.drop('fechapostulacion',axis=1)
tupla_postulada['y'] = 1
tupla_postulada = feature_genero_edad(tupla_postulada,gente_postulada)
tupla_postulada = tupla_postulada.drop('fechanacimiento',axis=1)
tupla_postulada = oneHotEncoding(tupla_postulada,'sexo')

In [44]:
tupla_postulada.head()

,idaviso,idpostulante,y,edad,FEM,MASC,NO_DECLARA
0,1112257047,NM5M,1,48,1,0,0
1,1111920714,NM5M,1,48,1,0,0
2,1112346945,NM5M,1,48,1,0,0
3,1112345547,NM5M,1,48,1,0,0
4,1112237522,5awk,1,56,1,0,0


In [45]:
tupla_postulada.shape

(6604534, 7)

# Gente no postulada

In [46]:
sample = generador_de_sample_serie(avisos_detalle['idaviso'],gente_no_postulada['idpostulante'],100,100)
sample['y'] = 0
sample = feature_genero_edad(sample,gente_no_postulada)
sample = sample.drop('fechanacimiento',axis=1)
sample = oneHotEncoding(sample,'sexo')

In [47]:
sample.head()

,idaviso,idpostulante,y,edad,FEM,MASC,NO_DECLARA
0,1112390041,GNZ8JWK,0,55,0,1,0
1,1112390041,ZDlz5E1,0,21,1,0,0
2,1112390041,a0N4q,0,43,1,0,0
3,1112390041,ep41j9,0,32,0,1,0
4,1112390041,awOL1m,0,34,1,0,0


# Test Final

In [48]:
test_final = feature_genero_edad(test_final,gente_postulada.append(gente_no_postulada))

In [49]:
# Salvo datos nulos
test_final.loc[test_final['sexo'].isnull(),'sexo'] = 'NO_DECLARA'
test_final.loc[test_final['edad'].isnull(),'edad'] = int(postulantes_genero_y_edad['edad'].mean())

In [50]:
test_final.isnull().sum()

idaviso               0
idpostulante          0
fechanacimiento    2838
sexo                  0
edad                  0
dtype: int64

In [51]:
#test_final = test_final.drop(columns=['idaviso','idpostulante','fechanacimiento'])
test_final['edad'] = test_final['edad'].astype('int32')
test_final = oneHotEncoding(test_final,'sexo')

# ML

In [52]:
sample = sample.append(tupla_postulada.sample(n=1000000,random_state=0))

In [53]:
sample.head()

,idaviso,idpostulante,y,edad,FEM,MASC,NO_DECLARA
0,1112390041,GNZ8JWK,0,55,0,1,0
1,1112390041,ZDlz5E1,0,21,1,0,0
2,1112390041,a0N4q,0,43,1,0,0
3,1112390041,ep41j9,0,32,0,1,0
4,1112390041,awOL1m,0,34,1,0,0


In [54]:
sample.tail()

,idaviso,idpostulante,y,edad,FEM,MASC,NO_DECLARA
4019134,1112457433,RzMAmXo,1,24,1,0,0
1441298,1112306596,ekOJ0L9,1,31,0,1,0
1290650,1112262555,PmJExxE,1,29,1,0,0
4598828,1112296411,4rPXrkM,1,25,0,1,0
6441257,1112355579,xkPYo6z,1,42,0,1,0


In [55]:
sample.shape

(2000000, 7)

In [56]:
vistas = vistas.rename(columns={'idAviso': 'idaviso'})

In [57]:
vistas.isnull().sum()

idaviso         0
timestamp       0
idpostulante    0
dtype: int64

In [58]:
vistas['visto'] = 1
vistas = vistas.drop('timestamp',axis=1)

In [61]:
vistas.head()

,idaviso,timestamp,idpostulante,visto
0,1111780242,2018-02-23T13:38:13.187-0500,YjVJQ6Z,1
1,1112263876,2018-02-23T13:38:14.296-0500,BmVpYoR,1
2,1112327963,2018-02-23T13:38:14.329-0500,wVkBzZd,1
3,1112318643,2018-02-23T13:38:17.921-0500,OqmP9pv,1
4,1111903673,2018-02-23T13:38:18.973-0500,DrpbXDP,1


In [65]:
#Esto es lo que yo hago para ver que tuplas del sample están en vistas: Hago un merge sobre dos columnas.
#Las que dan Nan es porque no están, entonces las pongo en cero. Las que estan quedan en 1 porque asi estaba la
#columna de visto. Pueden probar el isin de ustedes si les gusta mas.
sample = sample.merge(vistas,how='left',on=['idaviso','idpostulante'])

In [66]:
sample.shape

(2000000, 8)

In [67]:
sample.isnull().sum()

idaviso               0
idpostulante          0
y                     0
edad                  0
FEM                   0
MASC                  0
NO_DECLARA            0
visto           1731009
dtype: int64

In [68]:
sample = sample.fillna(0)

In [69]:
sample.head(1)

,idaviso,idpostulante,y,edad,FEM,MASC,NO_DECLARA,visto
0,1112390041,GNZ8JWK,0,55,0,1,0,0.0


In [70]:
sample.visto.value_counts()

0.0    1731009
1.0     268991
Name: visto, dtype: int64

In [71]:
sample = sample.drop(['idaviso','idpostulante'],axis=1)

In [72]:
sample.head()

,y,edad,FEM,MASC,NO_DECLARA,visto
0,0,55,0,1,0,0.0
1,0,21,1,0,0,0.0
2,0,43,1,0,0,0.0
3,0,32,0,1,0,0.0
4,0,34,1,0,0,0.0


In [73]:
#Lo mismo que hice con el sample antes lo hago con el test_final
test_final = test_final.merge(vistas,how='left',on=['idaviso','idpostulante'])

In [74]:
test_final.isnull().sum()

idaviso                0
idpostulante           0
fechanacimiento     2838
edad                   0
FEM                    0
MASC                   0
NO_DECLARA             0
visto              65839
dtype: int64

In [75]:
test_final = test_final.fillna(0)

In [76]:
test_final.visto.value_counts()

0.0    65839
1.0    34161
Name: visto, dtype: int64

In [77]:
test_final = test_final.drop(['idaviso','idpostulante','fechanacimiento'],axis=1)

In [78]:
test_final.head()

,edad,FEM,MASC,NO_DECLARA,visto
0,42,1,0,0,0.0
1,31,0,1,0,0.0
2,36,1,0,0,0.0
3,69,0,1,0,0.0
4,32,1,0,0,0.0


In [79]:
#-----------------------------------------------------------------
# Modelo Random Forest Classifier
#-----------------------------------------------------------------
from sklearn.ensemble import RandomForestClassifier

In [80]:
train_label = []
train_data = []

train_label = sample.iloc[:, 0]
train_data = sample.iloc[:, sample.columns != 'y']

In [81]:
ModeloRFC = RandomForestClassifier(n_estimators=100,random_state=0, n_jobs=-1)

In [82]:
ModeloRFC.fit(train_data, train_label)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [83]:
result = ModeloRFC.predict(test_final)

In [84]:
result.size

100000

In [85]:
import csv

In [86]:
def write_submission(test_data, prediction, file_output):
    
    archivo_entrada = open(test_data)
    entrada_csv = csv.reader(archivo_entrada)
    next(entrada_csv, None)  # skip the headers

    archivo_salida = open(file_output, 'w')
    submit_csv = csv.writer(archivo_salida)
    submit_csv.writerow(['id', 'sepostulo'])

    for reg1, reg2  in zip(entrada_csv, prediction):
        linea = [reg1[0], round(reg2, 2)]
        submit_csv.writerow(linea)
    archivo_salida.close()

In [87]:
write_submission("../data/test_final_100k.csv", result, "../predicciones/RFC_conVistas.csv")

In [88]:
prueba = pd.read_csv('../predicciones/RFC_conVistas.csv')

In [89]:
prueba.sepostulo.value_counts()

0.0    56833
1.0    43167
Name: sepostulo, dtype: int64